# Configuration

- **Setting up your Azure Machine Learning services workspace and configuring your notebook library**

---


## Table of Contents

1. [Introduction](#Introduction)
    1. What is an Azure Machine Learning workspace
1. [Setup](#Setup)
    1. Azure subscription
    1. Azure ML SDK and other library installation
    1. Azure Container Instance registration
1. [Configure your Azure ML Workspace](#Configure%20your%20Azure%20ML%20workspace)
    1. Workspace parameters
    1. Access your workspace
    1. Create a new workspace
    1. Create compute resources
1. [Next steps](#Next%20steps)

---


## 1. Introduction

- This notebook configures your library of notebooks to connect to an Azure Machine Learning (ML) workspace.  In this case, a library contains all of the notebooks in the current folder and any nested folders.  You can configure this notebook library to use an existing workspace or create a new workspace.

- Typically you will need to run this notebook only once per notebook library as all other notebooks will use connection information that is written here.  If you want to redirect your notebook library to work with a different workspace, then you should re-run this notebook.

In this notebook you will
* Learn about getting an Azure subscription
* Specify your workspace parameters
* Access or create your workspace
* Add a default compute cluster for your workspace


### What is an Azure Machine Learning workspace

- An Azure ML Workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an Azure ML Workspace coordinates storage,databases, and compute resources providing added functionality for machine learning experimentation, deployment, inference, and the monitoring of deployed models.
---

## 2. Setup

- This section describes activities required before you can access any Azure ML services functionality.

### 2.1. Azure Subscription

- In order to create an Azure ML Workspace, first you need access to an Azure subscription.  An Azure subscription allows you to manage storage, compute, and other assets in the Azure cloud.  You can [create a new subscription](https://azure.microsoft.com/en-us/free/) or access existing subscription information from the [Azure portal](https://portal.azure.com).  Later in this notebook you will need information such as your subscription ID in order to create and access AML workspaces.

### 2.2. Azure ML SDK and other library installation

If you are running in your own environment, follow [SDK installation instructions](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-environment).  If you are running in Azure Notebooks or another Microsoft managed environment, the SDK is already installed.

Also install following libraries to your environment. Many of the example notebooks depend on them

```
(myenv) $ conda install -y matplotlib tqdm scikit-learn
```

Once installation is complete, the following cell checks the Azure ML SDK version:

In [1]:
%pip install azureml-core


   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.3 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.3 MB 1.7 MB/s eta 0:00:02
   ------------ --------------------------- 1.0/3.3 MB 2.0 MB/s eta 0:00:02
   ------------------ --------------------- 1.6/3.3 MB 2.0 MB/s eta 0:00:01
   ------------------------- -------------- 2.1/3.3 MB 2.1 MB/s eta 0:00:01
   ------------------------- -------------- 2.1/3.3 MB 2.1 MB/s eta 0:00:01
   ---------------------------------- ----- 2.9/3.3 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 3.3/3.3 MB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----------------------------- ---------- 0.8/1.1 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------------------ -----------------

In [2]:
import azureml.core

print("This notebook was created using version 1.56.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.56.0 of the Azure ML SDK
You are currently using version 1.59.0 of the Azure ML SDK


- If you are using an older version of the SDK then this notebook was created using, you should upgrade your SDK.

---

### 2.3. Azure Container Instance registration
- Azure Machine Learning uses of [Azure Container Instance (ACI)](https://azure.microsoft.com/services/container-instances) to deploy dev/test web services. An Azure subscription needs to be registered to use ACI.  If you or the subscription owner have not yet registered ACI on your subscription, you will need to use the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) and execute the following commands.  Note that if you ran through the AML [quickstart](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started) you have already registered ACI.


```shell
# check to see if ACI is already registered
(myenv) $ az provider show -n Microsoft.ContainerInstance -o table

# if ACI is not registered, run this command.
# note you need to be the subscription owner in order to execute this command successfully.
(myenv) $ az provider register -n Microsoft.ContainerInstance
```

---

## 3.0 Configure your Azure ML workspace

### **3.1 Workspace parameters**

- To use an AML Workspace, you will need to import the Azure ML SDK and supply the following information:
  * Your subscription id
  * A resource group name
  * (optional) The region that will host your workspace
  * A name for your workspace

- You can get your subscription ID from the [Azure portal](https://portal.azure.com).

- You will also need access to a [_resource group_](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-overview#resource-groups), which organizes Azure resources and provides a default region for the resources in a group.  You can see what resource groups to which you have access, or create a new one in the [Azure portal](https://portal.azure.com).  If you don't have a resource group, the create workspace command will create one for you using the name you provide.

- The region to host your workspace will be used if you are creating a new workspace.  You do not need to specify this if you are using an existing workspace. You can find the list of supported regions [here](https://azure.microsoft.com/en-us/global-infrastructure/services/?products=machine-learning-service).  You should pick a region that is close to your location or that contains your data.

- The name for your workspace is unique within the subscription and should be descriptive enough to discern among other AML Workspaces.  The subscription may be used only by you, or it may be used by your department or your entire enterprise, so choose a name that makes sense for your situation.

- The following cell allows you to specify your workspace parameters.  This cell uses the python method `os.getenv` to read values from environment variables which is useful for automation.  If no environment variable exists, the parameters will be set to the specified default values.  

- If you ran the Azure Machine Learning [quickstart](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started) in Azure Notebooks, you already have a configured workspace!  You can go to your Azure Machine Learning Getting Started library, view *config.json* file, and copy-paste the values for subscription ID, resource group and workspace name below.

Replace the default values in the cell below with your workspace parameters

In [3]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="88187483-a72c-4f6a-83e5-43f1855302e1")
resource_group = os.getenv("RESOURCE_GROUP", default="RG000-Central_RG")
workspace_name = os.getenv("WORKSPACE_NAME", default="WS000-CentralWorkSpace")
workspace_region = os.getenv("WORKSPACE_REGION", default="South Africa North")

---

### **3.2 Access your workspace**

- The following cell uses the Azure ML SDK to attempt to load the workspace specified by your parameters.  If this cell succeeds, your notebook library will be configured to access the workspace from all notebooks using the `Workspace.from_config()` method.  The cell can fail if the specified workspace doesn't exist or you don't have permissions to access it. 

In [6]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Workspace configuration succeeded. Skip the workspace creation steps below


---

### **3.3 Create a new workspace**

- If you don't have an existing workspace and are the owner of the subscription or resource group, you can create a new workspace.  If you don't have a resource group, the create workspace command will create one for you using the name you provide.

**Note**: As with other Azure services, there are limits on certain resources (for example AmlCompute quota) associated with the Azure ML service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

- This cell will create an Azure ML workspace for you in a subscription provided you have the correct permissions.

This will fail if:
* You do not have permission to create a workspace in the resource group
* You do not have permission to create a resource group if it's non-existing.
* You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources.

**Note**: A Basic workspace is created by default. If you would like to create an Enterprise workspace, please specify sku = 'enterprise'.
Please visit our [pricing page](https://azure.microsoft.com/en-us/pricing/details/machine-learning/) for more details on our Enterprise edition.


In [5]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                    subscription_id = subscription_id,
                    resource_group = resource_group, 
                    location = workspace_region,
                    create_resource_group = False,
                    sku = 'basic',
                    exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

Deploying KeyVault with name ws000cenkeyvaultfeb4310f.
Deploying StorageAccount with name ws000censtorage31add608c.
Deploying AppInsights with name ws000ceninsights970603b4.
Deployed AppInsights with name ws000ceninsights970603b4. Took 69.56 seconds.
Deploying Workspace with name WS000-CentralWorkSpace.
Deployed Workspace with name WS000-CentralWorkSpace. Took 57.89 seconds.


### **3.4 Create compute resources for your training experiments**

- Many of the sample notebooks use Azure ML managed compute (AmlCompute) to train models using a dynamically scalable pool of compute. In this section you will create default compute clusters for use by the other notebooks and any other operations you choose.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

- To create a cluster, you need to specify a compute configuration that specifies the type of machine to be used and the scalability behaviors.  Then you choose a name for the cluster that is unique within the workspace that can be used to address the cluster later.

The cluster parameters are:
* vm_size - this describes the virtual machine type and size used in the cluster.  All machines in the cluster are the same type.  You can get the list of vm sizes available in your region by using the CLI command

```shell
az vm list-skus -o tsv
```
* min_nodes - this sets the minimum size of the cluster.  If you set the minimum to 0 the cluster will shut down all nodes while not in use.  Setting this number to a value higher than 0 will allow for faster start-up times, but you will also be billed when the cluster is not in use.
* max_nodes - this sets the maximum size of the cluster.  Setting this to a larger number allows for more concurrency and a greater distributed processing of scale-out jobs.


- To create a **CPU** cluster now, run the cell below. The autoscale settings mean that the cluster will scale down to 0 nodes when inactive and up to 4 nodes when busy.

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                        min_nodes=0,
                                                        max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Creating new cpu-cluster
InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Create a URI file data asset

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '<supported-path>'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="<description>",
    name="<name>",
    version="<version>"
)

ml_client.data.create_or_update(my_data)

# Create a URI file data Folder

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '<supported-path>'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FOLDER,
    description="<description>",
    name="<name>",
    version='<version>'
)

ml_client.data.create_or_update(my_data)

Imagine you create a Python script you want to run as a job, and you set the value of the input parameter input_data to be the URI folder data asset (which points to multiple CSV files). You can read all CSV files in the folder and concatenate them, which you can do by including the following code in your Python script:

In [ ]:
import argparse
import glob
import pandas as pd

parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str)
args = parser.parse_args()

data_path = args.input_data
all_files = glob.glob(data_path + "/*.csv")
df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

# MLTable data asset with the Python SDK

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '<path-including-mltable-file>'

my_data = Data(
    path=my_path,
    type=AssetTypes.MLTABLE,
    description="<description>",
    name="<name>",
    version='<version>'
)

ml_client.data.create_or_update(my_data)

When you parse a MLTable data asset as input to a Python script you want to run as an Azure Machine Learning job, you can include the following code to read the data:

In [ ]:
import argparse
import mltable
import pandas

parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str)
args = parser.parse_args()

tbl = mltable.load(args.input_data)
df = tbl.to_pandas_dataframe()

print(df.head(10))